In [ ]:
!pip install --upgrade evaluate jiwer

In [ ]:
from evaluate import load

In [ ]:
wer_metric = load('wer')

In [ ]:
reference = "the cat sat on the mat"
prediction = "the cat sit on the"

In [ ]:
wer = wer_metric.compute(references=[reference], predictions=[prediction])

In [ ]:
wer

0.3333333333333333

In [ ]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

In [ ]:
prediction = " He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similarly is drawn from eating and its results occur most readily to the mind."

In [ ]:
normalizer = BasicTextNormalizer()
normalized_prediction = normalizer(prediction)
normalized_prediction

' he tells us that at this festive season of the year with christmas and roast beef looming before us similarly is drawn from eating and its results occur most readily to the mind '

In [ ]:
reference = "HE TELLS US THAT AT THIS FESTIVE SEASON OF THE YEAR WITH CHRISTMAS AND ROAST BEEF LOOMING BEFORE US SIMILES DRAWN FROM EATING AND ITS RESULTS OCCUR MOST READILY TO THE MIND"
normalized_referece = normalizer(reference)

wer = wer_metric.compute(
    references=[normalized_referece], predictions=[normalized_prediction]
)
wer

0.0625

In [ ]:
from transformers import pipeline
import torch

if torch.cuda.is_available():
  device = 'cuda:0'
  dtype  = torch.float16
else:
  device = 'cpu'
  dtype = torch.float32

pipe = pipeline(
    'automatic-speech-recognition',
    model='openai/whisper-small',
    torch_dtype=dtype,
    device=device
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

In [ ]:
common_voice_test = load_dataset(
    'mozilla-foundation/common_voice_13_0',
    'ta',
    split='test'
)

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 5370it [00:00, 53508.94it/s]
Reading metadata...: 11696it [00:00, 59233.40it/s]
Reading metadata...: 19032it [00:00, 65676.39it/s]
Reading metadata...: 26546it [00:00, 69408.38it/s]
Reading metadata...: 33488it [00:00, 69081.71it/s]
Reading metadata...: 43350it [00:00, 67172.90it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 11937it [00:00, 97371.60it/s]


Generating test split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 11973it [00:00, 96759.18it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 6845it [00:00, 68439.30it/s]
Reading metadata...: 13956it [00:00, 70008.08it/s]
Reading metadata...: 21793it [00:00, 73824.45it/s]
Reading metadata...: 29430it [00:00, 74818.41it/s]
Reading metadata...: 36915it [00:00, 74826.93it/s]
Reading metadata...: 44398it [00:00, 73377.45it/s]
Reading metadata...: 52222it [00:00, 74943.69it/s]
Reading metadata...: 62262it [00:00, 82971.38it/s]
Reading metadata...: 73855it [00:00, 93207.63it/s]
Reading metadata...: 87707it [00:01, 85173.07it/s] 


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 5621it [00:00, 57223.88it/s]


In [ ]:
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

all_predictions = []

for prediction in tqdm(
        pipe(
            KeyDataset(common_voice_test, "audio"),
            max_new_tokens=128,
            generate_kwargs={"task":
                             "transcribe",
                             "language":'tamil'
                             },
            batch_size=32,
        ),
        total=len(common_voice_test),
    ):
    all_predictions.append(prediction["text"])

100%|██████████| 11973/11973 [24:21<00:00,  8.19it/s]


In [ ]:
from evaluate import load

In [ ]:
wer_metric = load('wer')
wer_ortho = 100*wer_metric.compute(references=common_voice_test['sentence'], predictions=all_predictions)
wer_ortho

73.99950480211632

In [ ]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

In [ ]:
normalizer = BasicTextNormalizer()

all_pred_norm = [normalizer(ele) for ele in all_predictions]
all_ref_norm = [normalizer(ele) for ele in common_voice_test['sentence']]

all_pred_norm = [ele for ele in all_pred_norm if len(ele) > 0]
all_ref_norm = [ele for ele in all_ref_norm if len(ele)>0]

wer_ortho = 100*wer_metric.compute(references=all_ref_norm, predictions=all_pred_norm)
wer_ortho

28.974922941094466